# Distilling Justice

## Imports

In [1]:
!pip install captum transformers[torch]


In [2]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from transformers import (
    BertForSequenceClassification, BertTokenizer, BertConfig,
    DistilBertForSequenceClassification, DistilBertTokenizer,
    Trainer, TrainingArguments
)
from captum.attr import IntegratedGradients
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\rhrou\miniconda3\envs\torch-gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [ ]:
df = pd.read_csv("data/justice.csv")
df = df.dropna(subset=["facts", "first_party_winner"])
df["label"] = df["first_party_winner"].astype(int)

print(f"      Loaded: {df.shape[0]} cases, {df.shape[1]} columns")

# 2. Remove Missing Values
print(f"\n[2/6] Removing missing values...")
initial_count = df.shape[0]
df.dropna(inplace=True)
print(f"      Retained: {df.shape[0]} cases ({df.shape[0]/initial_count*100:.1f}%)")

# 4. Extract and Clean Text
print(f"\n[3/6] Preprocessing text data...")
df_nlp = df[[TEXT_COLUMN]].copy()

# Remove HTML tags
df_nlp[TEXT_COLUMN] = df_nlp[TEXT_COLUMN].str.replace(r'<[^<>]*>', '', regex=True)

# Apply text preprocessing
df_nlp['facts_clean'] = df_nlp[TEXT_COLUMN].apply(
    lambda x: preprocess_text(x, lowercase=LOWERCASE, normalize=NORMALIZE_UNICODE)


from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["facts"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

In [4]:
df.head()

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area,label
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN,1
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights,1
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process,1
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights,1
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment,1


In [5]:
# Dataset helper - standard pytorch procedure

# Wraps tokenized text and labels so PyTorch's dataloader can batch and feed them to the model during training
class JusticeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, model_type=None):
        self.encodings = encodings
        self.labels = labels
        self.model_type = model_type
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
         # DistilBERT and MiniLM don't use token_type_ids
        if self.model_type in ["Step1_DistilBERT", "Step4_MiniLM", "Step3_TinyBERT"]:
            item.pop("token_type_ids", None)
        return item

## Metrics

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    macro_f1 = f1_score(labels, preds, average="macro")
    return {
        "accuracy": acc,
        "precision_class0": precision[0],
        "recall_class0": recall[0],
        "f1_class0": f1[0],
        "precision_class1": precision[1],
        "recall_class1": recall[1],
        "f1_class1": f1[1],
        "macro_f1": macro_f1
    }

## Training Arguments

In [7]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50
)


Explanation Function

In [8]:
def explain_prediction(model, tokenizer, text, label=None):
    model.eval()
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    if label is None:
        with torch.no_grad():
            outputs = model(**inputs)
            label = outputs.logits.argmax(-1).item()

    def forward_func(embeddings, attention_mask=None):
        outputs = model(inputs_embeds=embeddings, attention_mask=attention_mask)
        return torch.softmax(outputs.logits, dim=1)[:, label]

    embeddings = model.get_input_embeddings()(inputs["input_ids"])
    ig = IntegratedGradients(forward_func)
    attributions, _ = ig.attribute(
        embeddings,
        additional_forward_args=(inputs["attention_mask"],),
        return_convergence_delta=True
    )

    scores = attributions.sum(dim=-1).squeeze(0).detach().numpy()
    return scores


## Model Configurations (Teacher + 5 Steps)

In [1]:
model_configs = {
    "Teacher_BERT": {
        "tokenizer": "bert-base-uncased",
        "model": lambda: BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    },
    "Step1_DistilBERT": {
        "tokenizer": "distilbert-base-uncased",
        "model": lambda: DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    },
    "Step2_Custom6x512": {
        "tokenizer": "bert-base-uncased",
        "model": lambda: BertForSequenceClassification(
            BertConfig(
                vocab_size=30522,
                hidden_size=512,
                num_hidden_layers=6,
                num_attention_heads=8,
                intermediate_size=2048,
                num_labels=2
            )
        )
    },
    "Step3_TinyBERT": {
        "tokenizer": "bert-base-uncased",
        "model": lambda: BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_512D", num_labels=2)
    },
    "Step4_MiniLM": {
        "tokenizer": "bert-base-uncased",
        "model": lambda: BertForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased", num_labels=2)
    },
    "Step5_Custom2x384": {
        "tokenizer": "bert-base-uncased",
        "model": lambda: BertForSequenceClassification(
            BertConfig(
                vocab_size=30522,
                hidden_size=384,
                num_hidden_layers=2,
                num_attention_heads=6,
                intermediate_size=1024,
                num_labels=2
            )
        )
    }
}


## Loop through Models

In [13]:
results = {}
teacher_vectors = {}

for name, cfg in model_configs.items():
    print(f"\n🔹 Training {name}...")

    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained(cfg["tokenizer"]) if "bert" in cfg["tokenizer"] else DistilBertTokenizer.from_pretrained(cfg["tokenizer"])

    # Tokenize
    train_enc = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
    val_enc = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
    train_ds = JusticeDataset(train_enc, train_labels, model_type=name)
    val_ds = JusticeDataset(val_enc, val_labels, model_type=name)

    # Load model
    model = cfg["model"]()

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    results[name] = metrics

    # Explanations: only store teacher’s attributions once
    if name == "Teacher_BERT":
        teacher_vectors = [explain_prediction(model, tokenizer, text) for text in val_texts[:50]]
    else:
        student_vectors = [explain_prediction(model, tokenizer, text) for text in val_texts[:50]]
        cos_sims = []
        for t_vec, text in zip(teacher_vectors, val_texts[:50]):
            s_vec = explain_prediction(model, tokenizer, text)
            sim = cosine_similarity([t_vec], [s_vec])[0][0]
            cos_sims.append(sim)
        results[name]["cosine_similarity"] = np.mean(cos_sims)


🔹 Training Teacher_BERT...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\rhrou\AppData\Local\Temp\ipykernel_72656\3283061721.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Class0,Recall Class0,F1 Class0,Precision Class1,Recall Class1,F1 Class1,Macro F1
1,0.685700,0.646660,0.650456,0.000000,0.000000,0.000000,0.650456,1.000000,0.788214,0.394107
2,0.651700,0.641966,0.651976,1.000000,0.004348,0.008658,0.651446,1.000000,0.788940,0.398799
3,0.593800,0.651419,0.636778,0.443038,0.152174,0.226537,0.663212,0.897196,0.762661,0.494599


c:\Users\rhrou\miniconda3\envs\torch-gpu\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

## Results Summary

In [ ]:
results_df = pd.DataFrame(results).T
print(results_df)